### Coursera Capstone Project Week 3

### Part 1 : Data extraction and cleaning

<h4>First import all required libraries</h4>

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup 
import requests
print("Libraries imported !")

Libraries imported !


<h4>We then import data through web scrapping using beautifulsoup library</h4>

In [2]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
data  = requests.get(url).text

soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table')

Data are wrapped into a dataframe.

In [3]:
postal_code = pd.DataFrame(columns=["Postal Code", "Borough", "Neighbourhood"])

for row in tables[0].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        code = col[0].text.strip()
        borough = col[1].text.strip()
        neighbourhood = col[2].text.strip()
        postal_code = postal_code.append({"Postal Code":code, "Borough":borough, "Neighbourhood":neighbourhood}, ignore_index=True)

print('The dataframe has {} rows and {} columns.'.format(postal_code.shape[0],
        postal_code.shape[1]))
postal_code.head()

The dataframe has 180 rows and 3 columns.


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We need to drop rows with "Not assigned" values in columns "borough" or "neighbourhood". We identify 77 rows in both columns with such values.

In [4]:
print(' We have {} "Not assigned" value in column "Borough".'.format(
     postal_code[postal_code["Borough"] == "Not assigned"].shape[0]))

 We have 77 "Not assigned" value in column "Borough".


We clean our data by replacing "Not assigned" by "NaN" and then using dropna method.

In [5]:
postal_code["Borough"].replace("Not assigned", np.nan, inplace = True)
postal_code.dropna(subset=["Borough"], axis=0, inplace=True)
postal_code.reset_index(drop = True, inplace = True)
print(' We have {} "Not assigned" value in column "Borough".'.format(
     postal_code[postal_code["Borough"] == np.nan].shape[0]))
print('The dataframe has {} rows and {} columns.'.format(postal_code.shape[0],
        postal_code.shape[1]))
postal_code.head()

 We have 0 "Not assigned" value in column "Borough".
The dataframe has 103 rows and 3 columns.


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Part 2: Adding coordinates
<h4>Import the CSV file needed for geographical coordinates</h4>

In [6]:
url_csv = 'https://cocl.us/Geospatial_data'
geo_coordinate = pd.read_csv(url_csv)
geo_coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Add 2 columns for latitude and longitude

In [7]:
postal_code.insert(3, "Latitude",np.nan)
postal_code.insert(4, "Longitude",np.nan)

We then transform our dataframe into a dictionary, using to_dict method. Since this only allows to get a dictionary with columns names as keys, we performed a slight transform by using the first column ("postal code") as index and then transposing to get a dictionary with postal code as key and lists with latitude and longitude as values.

In [8]:
geo_dico = geo_coordinate.set_index('Postal Code').T.to_dict('list')
geo_dico.keys()

dict_keys(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H', 'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B', 'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B', 'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N', 'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A', 'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M', 'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A', 'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M', 'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W', 'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P', 'M9R', 'M9V', 'M9W'])

In [9]:
for i in range(0,postal_code.shape[0]):
    key = postal_code.iloc[i,0]
    if key in geo_dico:
        latitude = geo_dico[key][0]
        longitude = geo_dico[key][1]
        postal_code.iloc[i,3]=latitude
        postal_code.iloc[i,4]=longitude
postal_code.head()      

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Checking the results for "Central Bay Street"

In [10]:
for i in range (0,103):
    if postal_code.iloc[i,2] == "Central Bay Street":
        print(postal_code.loc[i])

Postal Code                     M5G
Borough            Downtown Toronto
Neighbourhood    Central Bay Street
Latitude                     43.658
Longitude                  -79.3874
Name: 24, dtype: object


### Part 3: Analysis

<h4>Import the libraries required for analysis</h4>

In [11]:
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print("Libraries imported !")

Libraries imported !


Create a map centered on Toronto with all boroughs represented.

In [12]:
map_toronto_city = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(postal_code['Latitude'], postal_code['Longitude'], 
                                           postal_code['Borough'], postal_code['Postal Code']):
    label = '{}: {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_city)  
    
map_toronto_city


We focus our analysis on boroughs containing "Toronto" (which sum up for a total of 40).

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(postal_code['Borough'].unique()),
        postal_code.shape[0]
    )
)
postal_code['Borough'].value_counts()

The dataframe has 11 boroughs and 103 neighborhoods.


North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 4
Mississauga          1
Toronto/York         1
Name: Borough, dtype: int64

Map with Toronto boroughs only

In [14]:
toronto_data = postal_code[postal_code['Borough'].str.contains('Toronto', case = False)].reset_index(drop=True)
toronto_data.head()
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], 
                                           toronto_data['Longitude'], 
                                           toronto_data['Borough'],
                                           toronto_data['Postal Code']):
    label = '{}: {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Credential (not displayed for safety reasons)

In [32]:
CLIENT_ID = '****'
CLIENT_SECRET = '****'
VERSION = '20180605' 
LIMIT = 100 

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: ****
CLIENT_SECRET:****


<h4>Userdefine function to get all neighbourhoods data from Foursquare</h4>

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Import data using Foursquare API and wrap them into a dataframe named toronto_venues

In [18]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
print("Data imported !")

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Har

In [19]:
print('The dataframe has {} rows and {} columns.'.format(toronto_venues.shape[0],
        toronto_venues.shape[1]))
toronto_venues.head()

The dataframe has 1612 rows and 7 columns.


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


We then group each neighbourhood and count the number of venues per neighbourhood.

In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.groupby('Neighbourhood').count()

There are 238 uniques categories.


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37


We convert categorial variables into dummy variables so we can perform a frequency analysis.

In [21]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We group by neighbourhood and take the mean of frequency of occurrence of each category.

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393,0.0,0.016393


Top 5 venus for each neighbourhood

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1        Bakery  0.05
2  Cocktail Bar  0.05
3    Restaurant  0.03
4      Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3              Bakery  0.05
4  Italian Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2       Auto Workshop  0.06
3                Park  0.06
4          Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.19
1      Airport Lounge  0.12
2    Airport Terminal  0.12
3               Plane  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                venue  freq
0         Coffee Sho

We put these rankings (top10) into a dataframe.

In [24]:
num_top_venues = 10

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], 
                                                                           num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Performing Arts Venue,Pet Store,Nightclub,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Boat or Ferry,Boutique,Sculpture Garden,Airport Gate,Airport Food Court
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Burger Joint,Salad Place,Thai Restaurant,Bubble Tea Shop,Indian Restaurant


We get a first insight over our dataset by performing a k-cluster analysis with 4 clusters.

In [25]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(n_clusters=5, random_state=0)

We merge toronto_grouped with toronto_data to add latitude and longitude for each neighborhood so we can plot our cluster results into a folium map.

In [26]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), 
                                     on='Neighbourhood')

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can now analysis each cluster to determine the discriminant venue that distinguish each cluster.

Cluster 0: Multi facilities services

In [27]:
cluster_label = 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Restaurant,Athletics & Sports,Italian Restaurant,Candy Store,Baby Store,Coffee Shop
9,West Toronto,0,Pharmacy,Bakery,Wine Shop,Middle Eastern Restaurant,Music Venue,Park,Pool,Portuguese Restaurant,Café,Brewery
11,West Toronto,0,Bar,Vietnamese Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Café,Asian Restaurant,Men's Store,Cupcake Shop,Brewery,Record Shop
18,Central Toronto,0,Park,Swim School,Bus Line,Business Service,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
20,Toronto/York,0,Pizza Place,Grocery Store,Convenience Store,Brewery,Bus Line,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
21,Central Toronto,0,Pizza Place,Hotel,Department Store,Sandwich Place,Food & Drink Shop,Breakfast Spot,Park,Escape Room,Electronics Store,Eastern European Restaurant
23,West Toronto,0,Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Diner,Fast Food Restaurant,Bookstore,Flea Market,Cajun / Creole Restaurant,Speakeasy
28,Downtown Toronto,0,Café,Bookstore,Bar,Japanese Restaurant,Bakery,Dessert Shop,Poutine Place,Pub,College Arts Building,Sushi Restaurant
31,Downtown Toronto,0,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Gaming Cafe,Burger Joint,Farmers Market,Grocery Store,Park
33,Downtown Toronto,0,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Boat or Ferry,Boutique,Sculpture Garden,Airport Gate,Airport Food Court


Cluster 1: Sport/Outside venue

In [28]:
cluster_label = 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Park,Sushi Restaurant,Trail,Jewelry Store,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
30,Central Toronto,1,Park,Playground,Tennis Court,Trail,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
34,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


Cluster 2: Coffee shop/fast food service

In [29]:
cluster_label = 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Café,Beer Store,Shoe Store
1,Downtown Toronto,2,Coffee Shop,Gym,Sushi Restaurant,Diner,Yoga Studio,Italian Restaurant,Japanese Restaurant,Bar,Bank,Mexican Restaurant
2,Downtown Toronto,2,Clothing Store,Coffee Shop,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Hotel,Italian Restaurant,Middle Eastern Restaurant,Café,Bookstore
3,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Cosmetics Shop,Cocktail Bar,Gastropub,Department Store,Italian Restaurant,Lingerie Store,Hotel
5,Downtown Toronto,2,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Jazz Club
6,Downtown Toronto,2,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Burger Joint,Salad Place,Thai Restaurant,Bubble Tea Shop,Indian Restaurant
8,Downtown Toronto,2,Coffee Shop,Café,Clothing Store,Restaurant,Bakery,Deli / Bodega,Thai Restaurant,Gym,Hotel,Bookstore
10,Downtown Toronto,2,Coffee Shop,Aquarium,Café,Hotel,Brewery,Restaurant,Scenic Lookout,Sporting Goods Shop,Italian Restaurant,Fried Chicken Joint
12,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Brewery,Indian Restaurant
13,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,Seafood Restaurant,Salad Place,Breakfast Spot


Cluster 3: Neighbourhood

In [30]:
cluster_label = 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Neighborhood,Health Food Store,Pub,Trail,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant


Cluster 4: Garden/Home catering

In [31]:
cluster_label = 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Garden,Home Service,Fast Food Restaurant,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
